In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)

    x_prev = x_prior

    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
using Random

In [15]:
P = 1.0

Random.seed!(123)

n = 250
data = -5.0 .+ collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [16]:
function inference(data, x0, c, P)
    n = length(data)
    
    model, (x, y) = smoothing(n, x0, c, P);

    ms_buffer = Vector{Marginal}(undef, n)
    fe_buffer = nothing
    
    fe_subscription = subscribe!(score(BetheFreeEnergy(), model), (fe) -> fe_buffer = fe)
    ms_subscription = subscribe!(getmarginals(x), (ms) -> copyto!(ms_buffer, ms))
    
    update!(y, data)
    
    unsubscribe!(ms_subscription)
    unsubscribe!(fe_subscription)
    
    return ms_buffer, fe_buffer
end

inference (generic function with 1 method)

In [17]:
# c[1] is C
# c[2] is μ0
function f(c)
    x0_prior = NormalMeanVariance(c[2], 100.0)
    ms, fe = inference(data, x0_prior, c[1], P)
    return fe
end

f (generic function with 1 method)

In [18]:
using Optim

In [19]:
res = optimize(f, ones(2), GradientDescent(), Optim.Options(g_tol = 1e-3, iterations = 100, store_trace = true, show_trace = true))

Iter     Function value   Gradient norm 
     0     3.655789e+02     8.149754e+02
 * time: 0.02835702896118164
     1     3.653239e+02     5.997076e-02
 * time: 0.7150709629058838
     2     3.652238e+02     3.400217e+02
 * time: 2.343522071838379
     3     3.651794e+02     2.660345e-02
 * time: 2.5858850479125977
     4     3.651615e+02     1.517870e+02
 * time: 4.221963882446289
     5     3.651526e+02     1.309935e-02
 * time: 4.464051961898804
     6     3.651483e+02     7.474478e+01
 * time: 6.0780930519104
     7     3.651461e+02     6.495536e-03
 * time: 6.319588899612427
     8     3.651450e+02     3.697864e+01
 * time: 7.933961868286133
     9     3.651445e+02     3.026425e-03
 * time: 8.17712688446045
    10     3.651443e+02     1.650103e+01
 * time: 9.668828010559082
    11     3.651442e+02     1.959583e-03
 * time: 9.909400939941406
    12     3.651442e+02     8.163634e+00
 * time: 11.47982406616211
    13     3.651442e+02     1.649326e-03
 * time: 11.710011005401611
    1

 * Status: success

 * Candidate solution
    Final objective value:     3.651440e+02

 * Found with
    Algorithm:     Gradient Descent

 * Convergence measures
    |x - x'|               = 3.15e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 6.32e-07 ≰ 0.0e+00
    |f(x) - f(x')|         = 6.45e-06 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.77e-08 ≰ 0.0e+00
    |g(x)|                 = 1.44e-04 ≤ 1.0e-03

 * Work counters
    Seconds run:   15  (vs limit Inf)
    Iterations:    17
    f(x) calls:    128
    ∇f(x) calls:   128


In [20]:
res.minimizer # Real values are indeed (c = 1.0 and μ0 = -5.0)

2-element Array{Float64,1}:
  1.0006316018959704
 -4.983657748477323